In [1]:
import os
import sys

src_path = os.path.abspath('../..')
print(src_path)
sys.path.append(src_path)

/home/zw12/missing_data


In [2]:
from src.utils import create_directory, dump_pickle, raw_data_path, processed_data_path, set_seed

In [3]:
import csv
import json
from datetime import datetime, timedelta
from collections import Counter, OrderedDict
import random
import pandas as pd
import numpy as np
from itertools import chain

In [4]:
set_seed(seed=42)

In [5]:
input_path = os.path.join(raw_data_path, "physionet.org/files/MIMIC-IV/2.0/hosp")
output_path = os.path.join(processed_data_path, "mimic4")
create_directory(output_path)

## patients

In [6]:
patients = pd.read_csv(os.path.join(input_path, 'patients.csv'))
print(patients.shape)
patients.head()

(315460, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [7]:
patients['dod'] = pd.to_datetime(patients['dod'])

In [8]:
patients.subject_id.nunique()

315460

In [9]:
patients = patients[['subject_id', 'gender', 'anchor_age', 'anchor_year', 'dod']]
patients.head()

,subject_id,gender,anchor_age,anchor_year,dod
0,10000032,F,52,2180,2180-09-09
1,10000048,F,23,2126,NaT
2,10000068,F,19,2160,NaT
3,10000084,M,72,2160,2161-02-13
4,10000102,F,27,2136,NaT


## admissions

In [10]:
admissions = pd.read_csv(os.path.join(input_path, 'admissions.csv'))
print(admissions.shape)
admissions.head()

(454324, 15)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [11]:
admissions.hadm_id.nunique()

454324

In [12]:
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])
admissions.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [13]:
admissions = admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'hospital_expire_flag', 'race']]
admissions.head()

,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,race
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0,WHITE
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,0,WHITE
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0,WHITE
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,0,WHITE
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,0,WHITE


## merge patients and admissions

In [14]:
df = admissions.merge(patients, on='subject_id', how='inner')
df.head()

,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,race,gender,anchor_age,anchor_year,dod
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0,WHITE,F,52,2180,2180-09-09
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,0,WHITE,F,52,2180,2180-09-09
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0,WHITE,F,52,2180,2180-09-09
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,0,WHITE,F,52,2180,2180-09-09
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,0,WHITE,F,19,2160,NaT


In [15]:
df['admit_age'] = df.apply(lambda x: x.admittime.year - x.anchor_year + x.anchor_age, axis=1)
df["duration"] = (df.dischtime - df.admittime).dt.total_seconds() / 60

In [16]:
df = df.sort_values(['subject_id', 'admittime'], ascending=True).reset_index(drop=True)
df.head()

,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,race,gender,anchor_age,anchor_year,dod,admit_age,duration
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0,WHITE,F,52,2180,2180-09-09,52,1132.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,0,WHITE,F,52,2180,2180-09-09,52,1462.0
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,0,WHITE,F,52,2180,2180-09-09,52,3200.0
3,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0,WHITE,F,52,2180,2180-09-09,52,2526.0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,0,WHITE,F,19,2160,NaT,19,430.0


In [17]:
df['readmit_in_days'] = (df.groupby('subject_id').admittime.shift(-1) - df.dischtime).dt.days
df

,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,race,gender,anchor_age,anchor_year,dod,admit_age,duration,readmit_in_days
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0,WHITE,F,52,2180,2180-09-09,52,1132.0,50.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,0,WHITE,F,52,2180,2180-09-09,52,1462.0,25.0
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,0,WHITE,F,52,2180,2180-09-09,52,3200.0,11.0
3,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0,WHITE,F,52,2180,2180-09-09,52,2526.0,NaN
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,0,WHITE,F,19,2160,NaT,19,430.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
454319,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,0,WHITE,F,46,2147,NaT,46,24587.0,522.0
454320,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,0,WHITE,F,46,2147,NaT,48,14416.0,NaN
454321,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,0,WHITE,M,58,2164,2164-09-17,58,5028.0,44.0
454322,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,1,WHITE,M,58,2164,2164-09-17,58,10075.0,NaN


In [18]:
df['die_in_days'] = (df.dod - df.dischtime).dt.days
df

,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,race,gender,anchor_age,anchor_year,dod,admit_age,duration,readmit_in_days,die_in_days
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,0,WHITE,F,52,2180,2180-09-09,52,1132.0,50.0,124.0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,0,WHITE,F,52,2180,2180-09-09,52,1462.0,25.0,73.0
2,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,0,WHITE,F,52,2180,2180-09-09,52,3200.0,11.0,45.0
3,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,0,WHITE,F,52,2180,2180-09-09,52,2526.0,NaN,32.0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,0,WHITE,F,19,2160,NaT,19,430.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454319,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,0,WHITE,F,46,2147,NaT,46,24587.0,522.0,NaN
454320,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,0,WHITE,F,46,2147,NaT,48,14416.0,NaN,NaN
454321,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,0,WHITE,M,58,2164,2164-09-17,58,5028.0,44.0,50.0
454322,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,1,WHITE,M,58,2164,2164-09-17,58,10075.0,NaN,-1.0


In [19]:
admittime = df[["hadm_id", "admittime"]]
admittime.head()

,hadm_id,admittime
0,22595853,2180-05-06 22:23:00
1,22841357,2180-06-26 18:27:00
2,29079034,2180-07-23 12:35:00
3,25742920,2180-08-05 23:44:00
4,25022803,2160-03-03 23:16:00


In [20]:
df = df[["subject_id", "hadm_id", "hospital_expire_flag", "readmit_in_days", "die_in_days", "race", "gender", "admit_age", "duration"]]
df

,subject_id,hadm_id,hospital_expire_flag,readmit_in_days,die_in_days,race,gender,admit_age,duration
0,10000032,22595853,0,50.0,124.0,WHITE,F,52,1132.0
1,10000032,22841357,0,25.0,73.0,WHITE,F,52,1462.0
2,10000032,29079034,0,11.0,45.0,WHITE,F,52,3200.0
3,10000032,25742920,0,NaN,32.0,WHITE,F,52,2526.0
4,10000068,25022803,0,NaN,NaN,WHITE,F,19,430.0
...,...,...,...,...,...,...,...,...,...
454319,19999828,29734428,0,522.0,NaN,WHITE,F,46,24587.0
454320,19999828,25744818,0,NaN,NaN,WHITE,F,48,14416.0
454321,19999840,26071774,0,44.0,50.0,WHITE,M,58,5028.0
454322,19999840,21033226,1,NaN,-1.0,WHITE,M,58,10075.0


In [21]:
df.to_csv(os.path.join(output_path, 'patients_admissions_tmp.csv'), index=False)

## prescriptions

In [22]:
prescriptions_raw = pd.read_csv(os.path.join(input_path, 'prescriptions.csv'), dtype={'drug': str})
print(prescriptions_raw.shape)
prescriptions_raw.head()

/tmp/ipykernel_1865939/828167779.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions_raw = pd.read_csv(os.path.join(input_path, 'prescriptions.csv'), dtype={'drug': str})


(16219412, 20)


,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,APAP500,004490,9.041989e+08,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0.000000e+00,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,FURO40,008209,5.107901e+10,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,RALT400,063231,6.022761e+06,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,HEPA5I,006549,6.332303e+10,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


In [23]:
prescriptions = prescriptions_raw[['subject_id', 'hadm_id', 'starttime', 'drug']]
print(prescriptions.shape)
prescriptions.head()

(16219412, 4)


,subject_id,hadm_id,starttime,drug
0,10000032,22595853,2180-05-07 01:00:00,Acetaminophen
1,10000032,22595853,2180-05-07 00:00:00,Sodium Chloride 0.9% Flush
2,10000032,22595853,2180-05-08 08:00:00,Furosemide
3,10000032,22595853,2180-05-07 01:00:00,Raltegravir
4,10000032,22595853,2180-05-07 00:00:00,Heparin


In [24]:
prescriptions = prescriptions.merge(admittime, on="hadm_id", how="inner")
prescriptions['starttime'] = pd.to_datetime(prescriptions['starttime'])
prescriptions["timestamp"] = (prescriptions.starttime - prescriptions.admittime).dt.total_seconds()/60
prescriptions

,subject_id,hadm_id,starttime,drug,admittime,timestamp
0,10000032,22595853,2180-05-07 01:00:00,Acetaminophen,2180-05-06 22:23:00,157.0
1,10000032,22595853,2180-05-07 00:00:00,Sodium Chloride 0.9% Flush,2180-05-06 22:23:00,97.0
2,10000032,22595853,2180-05-08 08:00:00,Furosemide,2180-05-06 22:23:00,2017.0
3,10000032,22595853,2180-05-07 01:00:00,Raltegravir,2180-05-06 22:23:00,157.0
4,10000032,22595853,2180-05-07 00:00:00,Heparin,2180-05-06 22:23:00,97.0
...,...,...,...,...,...,...
16219407,19999987,23865745,2145-11-03 00:00:00,Influenza Virus Vaccine,2145-11-02 21:38:00,142.0
16219408,19999987,23865745,2145-11-04 10:00:00,Levothyroxine Sodium,2145-11-02 21:38:00,2182.0
16219409,19999987,23865745,2145-11-10 10:00:00,Duloxetine,2145-11-02 21:38:00,10822.0
16219410,19999987,23865745,2145-11-09 10:00:00,Venlafaxine XR,2145-11-02 21:38:00,9382.0


In [25]:
prescriptions = prescriptions[["subject_id", "hadm_id", "timestamp", "drug"]]
prescriptions

,subject_id,hadm_id,timestamp,drug
0,10000032,22595853,157.0,Acetaminophen
1,10000032,22595853,97.0,Sodium Chloride 0.9% Flush
2,10000032,22595853,2017.0,Furosemide
3,10000032,22595853,157.0,Raltegravir
4,10000032,22595853,97.0,Heparin
...,...,...,...,...
16219407,19999987,23865745,142.0,Influenza Virus Vaccine
16219408,19999987,23865745,2182.0,Levothyroxine Sodium
16219409,19999987,23865745,10822.0,Duloxetine
16219410,19999987,23865745,9382.0,Venlafaxine XR


In [26]:
prescriptions = prescriptions.dropna()
prescriptions = prescriptions.drop_duplicates()
print(prescriptions.drug.nunique())
print(prescriptions.shape)

9790
(14370026, 4)


In [27]:
prescriptions = prescriptions.sort_values(['subject_id', 'hadm_id', 'timestamp', 'drug'], ascending=True).reset_index(drop=True)
prescriptions

,subject_id,hadm_id,timestamp,drug
0,10000032,22595853,97.0,Heparin
1,10000032,22595853,97.0,Influenza Vaccine Quadrivalent
2,10000032,22595853,97.0,Sodium Chloride 0.9% Flush
3,10000032,22595853,157.0,Acetaminophen
4,10000032,22595853,157.0,Albuterol Inhaler
...,...,...,...,...
14370021,19999987,23865745,9982.0,LeVETiracetam
14370022,19999987,23865745,9982.0,Sulfameth/Trimethoprim DS
14370023,19999987,23865745,10822.0,Duloxetine
14370024,19999987,23865745,10822.0,Venlafaxine XR


In [28]:
prescriptions.to_csv(os.path.join(output_path, 'prescriptions_tmp.csv'), index=False)

## diagnoses_icd

In [29]:
diagnoses_icd = pd.read_csv(os.path.join(input_path, 'diagnoses_icd.csv'))
print(diagnoses_icd.shape)
diagnoses_icd.head()

(5006884, 5)


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [30]:
d_icd_diagnoses = pd.read_csv(os.path.join(input_path, 'd_icd_diagnoses.csv'))
print(d_icd_diagnoses.shape)
d_icd_diagnoses.head()

(109775, 3)


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [31]:
diagnoses_icd = diagnoses_icd.merge(d_icd_diagnoses, on=["icd_code", "icd_version"])
diagnoses_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,10000032,22595853,1,5723,9,Portal hypertension
1,10000826,20032235,4,5723,9,Portal hypertension
2,10000826,28289260,1,5723,9,Portal hypertension
3,10005866,26158160,4,5723,9,Portal hypertension
4,10008924,23676183,7,5723,9,Portal hypertension
...,...,...,...,...,...,...
5006879,19990427,29695607,24,T24011A,10,"Burn of unspecified degree of right thigh, ini..."
5006880,19996016,28015466,4,O30093,10,"Twin pregnancy, unable to determine number of ..."
5006881,19996783,25894657,20,H353131,10,"Nonexudative age-related macular degeneration,..."
5006882,19997062,20096107,1,K8036,10,Calculus of bile duct with acute and chronic c...


In [32]:
diagnoses_icd = diagnoses_icd.dropna()
diagnoses_icd = diagnoses_icd.drop_duplicates()
print(diagnoses_icd.long_title.nunique())
print(diagnoses_icd.shape)

24846
(5006884, 6)


In [33]:
diagnoses_icd = diagnoses_icd.sort_values(['subject_id', 'hadm_id', 'seq_num'], ascending=True).reset_index(drop=True)
diagnoses_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,10000032,22595853,1,5723,9,Portal hypertension
1,10000032,22595853,2,78959,9,Other ascites
2,10000032,22595853,3,5715,9,Cirrhosis of liver without mention of alcohol
3,10000032,22595853,4,07070,9,Unspecified viral hepatitis C without hepatic ...
4,10000032,22595853,5,496,9,"Chronic airway obstruction, not elsewhere clas..."
...,...,...,...,...,...,...
5006879,19999987,23865745,7,41401,9,Coronary atherosclerosis of native coronary ar...
5006880,19999987,23865745,8,78039,9,Other convulsions
5006881,19999987,23865745,9,0413,9,Friedländer's bacillus infection in conditions...
5006882,19999987,23865745,10,36846,9,Homonymous bilateral field defects


In [34]:
diagnoses_icd = diagnoses_icd[["subject_id", "hadm_id", "long_title"]]
diagnoses_icd

,subject_id,hadm_id,long_title
0,10000032,22595853,Portal hypertension
1,10000032,22595853,Other ascites
2,10000032,22595853,Cirrhosis of liver without mention of alcohol
3,10000032,22595853,Unspecified viral hepatitis C without hepatic ...
4,10000032,22595853,"Chronic airway obstruction, not elsewhere clas..."
...,...,...,...
5006879,19999987,23865745,Coronary atherosclerosis of native coronary ar...
5006880,19999987,23865745,Other convulsions
5006881,19999987,23865745,Friedländer's bacillus infection in conditions...
5006882,19999987,23865745,Homonymous bilateral field defects


In [35]:
diagnoses_icd.to_csv(os.path.join(output_path, 'diagnoses_icd_tmp.csv'), index=False)

## procedures_icd

In [36]:
procedures_icd = pd.read_csv(os.path.join(input_path, 'procedures_icd.csv'))
print(procedures_icd.shape)
procedures_icd.head()

(704124, 6)


,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


In [37]:
d_icd_procedures = pd.read_csv(os.path.join(input_path, 'd_icd_procedures.csv'))
print(d_icd_procedures.shape)
d_icd_procedures.head()

(85257, 3)


,icd_code,icd_version,long_title
0,0001,9,Therapeutic ultrasound of vessels of head and ...
1,0002,9,Therapeutic ultrasound of heart
2,0003,9,Therapeutic ultrasound of peripheral vascular ...
3,0009,9,Other therapeutic ultrasound
4,001,10,"Central Nervous System and Cranial Nerves, Bypass"


In [38]:
procedures_icd = procedures_icd.merge(d_icd_procedures, on=["icd_code", "icd_version"])
procedures_icd

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,long_title
0,10000032,22595853,1,2180-05-07,5491,9,Percutaneous abdominal drainage
1,10000032,22841357,1,2180-06-27,5491,9,Percutaneous abdominal drainage
2,10000032,25742920,1,2180-08-06,5491,9,Percutaneous abdominal drainage
3,10000826,20032235,1,2146-12-05,5491,9,Percutaneous abdominal drainage
4,10000826,20032235,2,2146-12-11,5491,9,Percutaneous abdominal drainage
...,...,...,...,...,...,...,...
704119,19988632,21153934,6,2182-05-25,0PUR07Z,10,Supplement Right Thumb Phalanx with Autologous...
704120,19989918,26299748,1,2182-04-24,00HE0MZ,10,Insertion of Neurostimulator Lead into Cranial...
704121,19990398,28149711,1,2184-03-10,0PHS34Z,10,Insertion of Internal Fixation Device into Lef...
704122,19997538,22701415,3,2168-05-01,0TQ74ZZ,10,"Repair Left Ureter, Percutaneous Endoscopic Ap..."


In [39]:
procedures_icd = procedures_icd.dropna()
procedures_icd = procedures_icd.drop_duplicates()
print(procedures_icd.icd_code.nunique())
print(procedures_icd.shape)

12788
(704124, 7)


In [40]:
procedures_icd = procedures_icd.merge(admittime, on="hadm_id", how="inner")
procedures_icd['chartdate'] = pd.to_datetime(procedures_icd['chartdate'])
procedures_icd["timestamp"] = (procedures_icd.chartdate - procedures_icd.admittime).dt.total_seconds()/60
procedures_icd

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,long_title,admittime,timestamp
0,10000032,22595853,1,2180-05-07,5491,9,Percutaneous abdominal drainage,2180-05-06 22:23:00,97.0
1,10000032,22841357,1,2180-06-27,5491,9,Percutaneous abdominal drainage,2180-06-26 18:27:00,333.0
2,10000032,25742920,1,2180-08-06,5491,9,Percutaneous abdominal drainage,2180-08-05 23:44:00,16.0
3,10000826,20032235,1,2146-12-05,5491,9,Percutaneous abdominal drainage,2146-12-05 19:07:00,-1147.0
4,10000826,20032235,2,2146-12-11,5491,9,Percutaneous abdominal drainage,2146-12-05 19:07:00,7493.0
...,...,...,...,...,...,...,...,...,...
704119,19884857,24110971,2,2186-07-31,0D7Q4ZZ,10,"Dilation of Anus, Percutaneous Endoscopic Appr...",2186-07-31 00:00:00,0.0
704120,19885726,28337917,1,2119-01-11,0F924ZZ,10,"Drainage of Left Lobe Liver, Percutaneous Endo...",2119-01-11 19:12:00,-1152.0
704121,19901064,25228731,1,2111-02-22,2W3CX3Z,10,Immobilization of Right Lower Arm using Brace,2111-02-22 07:59:00,-479.0
704122,19939336,28624200,1,2141-10-17,0QPFX5Z,10,Removal of External Fixation Device from Left ...,2141-10-08 11:49:00,12251.0


In [41]:
procedures_icd = procedures_icd.sort_values(['subject_id', 'hadm_id', 'timestamp', 'seq_num'], ascending=True).reset_index(drop=True)
procedures_icd

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,long_title,admittime,timestamp
0,10000032,22595853,1,2180-05-07,5491,9,Percutaneous abdominal drainage,2180-05-06 22:23:00,97.0
1,10000032,22841357,1,2180-06-27,5491,9,Percutaneous abdominal drainage,2180-06-26 18:27:00,333.0
2,10000032,25742920,1,2180-08-06,5491,9,Percutaneous abdominal drainage,2180-08-05 23:44:00,16.0
3,10000068,25022803,1,2160-03-03,8938,9,Other nonoperative respiratory measurements,2160-03-03 23:16:00,-1396.0
4,10000117,27988844,1,2183-09-19,0QS734Z,10,Reposition Left Upper Femur with Internal Fixa...,2183-09-18 18:10:00,350.0
...,...,...,...,...,...,...,...,...,...
704119,19999840,21033226,5,2164-09-16,0331,9,Spinal tap,2164-09-10 13:47:00,7813.0
704120,19999840,26071774,1,2164-07-25,8891,9,Magnetic resonance imaging of brain and brain ...,2164-07-25 00:27:00,-27.0
704121,19999840,26071774,2,2164-07-25,8841,9,Arteriography of cerebral arteries,2164-07-25 00:27:00,-27.0
704122,19999987,23865745,2,2145-11-02,9671,9,Continuous invasive mechanical ventilation for...,2145-11-02 21:38:00,-1298.0


In [42]:
procedures_icd = procedures_icd[["subject_id", "hadm_id", "timestamp", "long_title"]]
procedures_icd

,subject_id,hadm_id,timestamp,long_title
0,10000032,22595853,97.0,Percutaneous abdominal drainage
1,10000032,22841357,333.0,Percutaneous abdominal drainage
2,10000032,25742920,16.0,Percutaneous abdominal drainage
3,10000068,25022803,-1396.0,Other nonoperative respiratory measurements
4,10000117,27988844,350.0,Reposition Left Upper Femur with Internal Fixa...
...,...,...,...,...
704119,19999840,21033226,7813.0,Spinal tap
704120,19999840,26071774,-27.0,Magnetic resonance imaging of brain and brain ...
704121,19999840,26071774,-27.0,Arteriography of cerebral arteries
704122,19999987,23865745,-1298.0,Continuous invasive mechanical ventilation for...


In [43]:
procedures_icd.to_csv(os.path.join(output_path, 'procedures_icd_tmp.csv'), index=False)

## labevents

In [44]:
selected_labs = [
    'Hematocrit', 
    'Platelet', 
    'WBC', 
    'Bilirubin', 
    'pH', 
    'Bicarbonate', 
    'Creatinine', 
    'Lactate', 
    'Potassium', 
    'Sodium', 
]

In [45]:
d_labitems = pd.read_csv(os.path.join(input_path, 'd_labitems.csv'))
print(d_labitems.shape)
d_labitems.head()

(1623, 4)


,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas


In [46]:
d_labitems = d_labitems.dropna(subset="label")

In [47]:
d_labitems = d_labitems[d_labitems.label.apply(lambda x: any(lab in x for lab in selected_labs))]
d_labitems

,itemid,label,fluid,category
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
8,50810,"Hematocrit, Calculated",Blood,Blood Gas
11,50813,Lactate,Blood,Blood Gas
18,50820,pH,Blood,Blood Gas
20,50822,"Potassium, Whole Blood",Blood,Blood Gas
...,...,...,...,...
1531,52610,Potassium,Blood,Chemistry
1532,52623,Sodium,Blood,Chemistry
1537,52730,pH,Urine,Chemistry
1565,53089,"Bilirubin, Total",Blood,Chemistry


In [48]:
labevents_raw = pd.read_csv(os.path.join(input_path, 'labevents.csv'))
print(labevents_raw.shape)
labevents_raw.head()

(124342638, 15)


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN
1,2,10000032,NaN,45421181,51274,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN
4,5,10000032,NaN,52958335,50862,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN


In [49]:
labevents = labevents_raw[['subject_id', 'hadm_id', 'specimen_id', 'storetime', 'itemid', 'valuenum', 'flag']]
print(labevents.shape)
labevents.head()

(124342638, 7)


,subject_id,hadm_id,specimen_id,storetime,itemid,valuenum,flag
0,10000032,NaN,45421181,2180-03-23 15:15:00,51237,1.4,abnormal
1,10000032,NaN,45421181,2180-03-23 15:15:00,51274,15.1,abnormal
2,10000032,NaN,52958335,2180-03-25 11:06:00,50853,15.0,abnormal
3,10000032,NaN,52958335,2180-03-23 16:40:00,50861,102.0,abnormal
4,10000032,NaN,52958335,2180-03-23 16:40:00,50862,3.3,abnormal


In [50]:
labevents = labevents.merge(d_labitems, on="itemid", how="inner")
print(labevents.shape)
labevents.head()

(26406234, 10)


,subject_id,hadm_id,specimen_id,storetime,itemid,valuenum,flag,label,fluid,category
0,10000032,NaN,52958335,2180-03-23 16:40:00,50882,27.0,NaN,Bicarbonate,Blood,Chemistry
1,10000032,NaN,43430170,2180-05-06 23:16:00,50882,27.0,NaN,Bicarbonate,Blood,Chemistry
2,10000032,22595853.0,93044861,2180-05-07 07:03:00,50882,28.0,NaN,Bicarbonate,Blood,Chemistry
3,10000032,NaN,41498752,2180-06-03 13:04:00,50882,28.0,NaN,Bicarbonate,Blood,Chemistry
4,10000032,NaN,86302346,2180-06-03 13:04:00,50882,29.0,NaN,Bicarbonate,Blood,Chemistry


In [51]:
labevents = labevents.merge(admittime, on="hadm_id", how="inner")
labevents['storetime'] = pd.to_datetime(labevents['storetime'])
labevents["timestamp"] = (labevents.storetime - labevents.admittime).dt.total_seconds()/60
labevents

,subject_id,hadm_id,specimen_id,storetime,itemid,valuenum,flag,label,fluid,category,admittime,timestamp
0,10000032,22595853.0,93044861,2180-05-07 07:03:00,50882,28.0,NaN,Bicarbonate,Blood,Chemistry,2180-05-06 22:23:00,520.0
1,10000032,22595853.0,93044861,2180-05-07 07:03:00,50885,1.3,NaN,"Bilirubin, Total",Blood,Chemistry,2180-05-06 22:23:00,520.0
2,10000032,22595853.0,93044861,2180-05-07 07:03:00,50912,0.3,abnormal,Creatinine,Blood,Chemistry,2180-05-06 22:23:00,520.0
3,10000032,22595853.0,93044861,2180-05-07 07:03:00,50971,4.5,NaN,Potassium,Blood,Chemistry,2180-05-06 22:23:00,520.0
4,10000032,22595853.0,93044861,2180-05-07 07:03:00,50983,137.0,NaN,Sodium,Blood,Chemistry,2180-05-06 22:23:00,520.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15040550,17004414,24547596.0,5933004,2130-05-24 14:04:00,51369,60.0,abnormal,"Hematocrit, Joint Fluid",Joint Fluid,Hematology,2130-05-23 23:37:00,867.0
15040551,19578341,26376851.0,86858804,2200-04-03 00:13:00,51369,NaN,NaN,"Hematocrit, Joint Fluid",Joint Fluid,Hematology,2200-04-03 02:37:00,-144.0
15040552,15429668,24107779.0,42824969,2112-07-13 17:48:00,51795,18.0,NaN,"Lactate Dehydrogenase, CSF",Cerebrospinal Fluid,Chemistry,2112-07-12 18:03:00,1425.0
15040553,18436961,20120465.0,28999983,2159-03-14 21:39:00,51064,6.1,NaN,"Potassium, Stool",Stool,Chemistry,2159-03-14 14:07:00,452.0


In [52]:
labevents = labevents[["subject_id", "hadm_id", "timestamp", "itemid", "valuenum", "flag"]]
labevents

,subject_id,hadm_id,timestamp,itemid,valuenum,flag
0,10000032,22595853.0,520.0,50882,28.0,NaN
1,10000032,22595853.0,520.0,50885,1.3,NaN
2,10000032,22595853.0,520.0,50912,0.3,abnormal
3,10000032,22595853.0,520.0,50971,4.5,NaN
4,10000032,22595853.0,520.0,50983,137.0,NaN
...,...,...,...,...,...,...
15040550,17004414,24547596.0,867.0,51369,60.0,abnormal
15040551,19578341,26376851.0,-144.0,51369,NaN,NaN
15040552,15429668,24107779.0,1425.0,51795,18.0,NaN
15040553,18436961,20120465.0,452.0,51064,6.1,NaN


In [53]:
labevents = labevents.dropna(subset=["subject_id", "hadm_id", "timestamp", "itemid", "valuenum"])
labevents = labevents.drop_duplicates()
print(labevents.shape)

(14606360, 6)


In [54]:
labevents = labevents.sort_values(['subject_id', 'hadm_id', 'timestamp', 'itemid'], ascending=True).reset_index(drop=True)
labevents

,subject_id,hadm_id,timestamp,itemid,valuenum,flag
0,10000032,22595853.0,137.0,51491,6.0,NaN
1,10000032,22595853.0,479.0,51221,37.6,NaN
2,10000032,22595853.0,479.0,51265,71.0,abnormal
3,10000032,22595853.0,520.0,50882,28.0,NaN
4,10000032,22595853.0,520.0,50885,1.3,NaN
...,...,...,...,...,...,...
14606355,19999987,23865745.0,9201.0,50912,0.9,NaN
14606356,19999987,23865745.0,9201.0,50971,3.4,NaN
14606357,19999987,23865745.0,9201.0,50983,144.0,NaN
14606358,19999987,23865745.0,9208.0,51221,36.6,NaN


In [55]:
labevents.groupby(["subject_id", "hadm_id"]).timestamp.nunique().mean()

15.067257219647313

In [56]:
labevents.to_csv(os.path.join(output_path, 'labevents_tmp.csv'), index=False)

## discharge

In [57]:
note_input_path = os.path.join(raw_data_path, "physionet.org/files//mimic-iv-note/2.2/note")
discharge = pd.read_csv(os.path.join(note_input_path, 'discharge.csv.gz'))
print(discharge.shape)
discharge.head()

(331794, 8)


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [58]:
def extract_discharge_instructions(note):
    # Find the start and end indices of the Discharge Instructions section
    start_index = note.find("Discharge Instructions:")
    end_index = note.find("Followup Instructions:")
    
    # If both indices are found, extract the section
    if start_index != -1 and end_index != -1:
        instructions = note[start_index + len("Discharge Instructions:"):end_index].strip()
    else:
        return float('nan')
    
    # Clean up the text for NLP tasks
    instructions = instructions.replace("___", "").replace("\n", " ").strip()
    
    return instructions

In [59]:
discharge.text = discharge.text.apply(extract_discharge_instructions)
discharge

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,"Dear Ms. , It was a pleasure taking care of yo..."
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,"Dear , was a pleasure to take care of you at..."
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,"Dear Ms. , You were admitted to because of c..."
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,"Dear Ms. , It was a pleasure caring for you d..."
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,"Dear Mr. , It was a privilege caring for you..."
...,...,...,...,...,...,...,...,...
331789,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,"Dear Ms. , You were admitted to with a larg..."
331790,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,"Dear Ms. , WHY YOU CAME TO THE HOSPITAL? You ..."
331791,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,You were admitted to this hospital because you...
331792,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,pt expired


In [60]:
discharge = discharge[["subject_id", "hadm_id", "text"]]
discharge

,subject_id,hadm_id,text
0,10000032,22595853,"Dear Ms. , It was a pleasure taking care of yo..."
1,10000032,22841357,"Dear , was a pleasure to take care of you at..."
2,10000032,29079034,"Dear Ms. , You were admitted to because of c..."
3,10000032,25742920,"Dear Ms. , It was a pleasure caring for you d..."
4,10000084,23052089,"Dear Mr. , It was a privilege caring for you..."
...,...,...,...
331789,19999828,29734428,"Dear Ms. , You were admitted to with a larg..."
331790,19999828,25744818,"Dear Ms. , WHY YOU CAME TO THE HOSPITAL? You ..."
331791,19999840,26071774,You were admitted to this hospital because you...
331792,19999840,21033226,pt expired


In [61]:
discharge = discharge.dropna()
discharge

,subject_id,hadm_id,text
0,10000032,22595853,"Dear Ms. , It was a pleasure taking care of yo..."
1,10000032,22841357,"Dear , was a pleasure to take care of you at..."
2,10000032,29079034,"Dear Ms. , You were admitted to because of c..."
3,10000032,25742920,"Dear Ms. , It was a pleasure caring for you d..."
4,10000084,23052089,"Dear Mr. , It was a privilege caring for you..."
...,...,...,...
331789,19999828,29734428,"Dear Ms. , You were admitted to with a larg..."
331790,19999828,25744818,"Dear Ms. , WHY YOU CAME TO THE HOSPITAL? You ..."
331791,19999840,26071774,You were admitted to this hospital because you...
331792,19999840,21033226,pt expired


In [62]:
discharge.to_csv(os.path.join(output_path, 'discharge_tmp.csv'), index=False)

## discharge v2

In [6]:
note_input_path = os.path.join(raw_data_path, "physionet.org/files//mimic-iv-note/2.2/note")
discharge = pd.read_csv(os.path.join(note_input_path, 'discharge.csv.gz'))
print(discharge.shape)
discharge.head()

(331794, 8)


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...


In [7]:
def extract_brief_hospital_course(note):
    # Find the start and end indices of the Discharge Instructions section
    start_index = note.find("Brief Hospital Course:")
    end_index = note.find("Medications on Admission:")
    
    # If both indices are found, extract the section
    if start_index != -1 and end_index != -1:
        instructions = note[start_index + len("Brief Hospital Course:"):end_index].strip()
    else:
        return float('nan')
    
    # Clean up the text for NLP tasks
    instructions = instructions.replace("___", "").replace("\n", " ").strip()
    
    return instructions

In [8]:
discharge.text = discharge.text.apply(extract_brief_hospital_course)
discharge

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVD..."
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,"with HIV on HAART, HCV cirrhosis with ascites ..."
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,NaN
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,NaN
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,NaN
...,...,...,...,...,...,...,...,...
331789,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,Ms. is a year old female with a history of m...
331790,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,NaN
331791,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,year old man hx of epilepsy and left temporal ...
331792,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,Mr. was admitted with status epilepticus. Hi...


In [9]:
discharge = discharge[["subject_id", "hadm_id", "text"]]
discharge

,subject_id,hadm_id,text
0,10000032,22595853,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVD..."
1,10000032,22841357,"with HIV on HAART, HCV cirrhosis with ascites ..."
2,10000032,29079034,NaN
3,10000032,25742920,NaN
4,10000084,23052089,NaN
...,...,...,...
331789,19999828,29734428,Ms. is a year old female with a history of m...
331790,19999828,25744818,NaN
331791,19999840,26071774,year old man hx of epilepsy and left temporal ...
331792,19999840,21033226,Mr. was admitted with status epilepticus. Hi...


In [10]:
discharge = discharge.dropna()
discharge

,subject_id,hadm_id,text
0,10000032,22595853,"HCV cirrhosis c/b ascites, hiv on ART, h/o IVD..."
1,10000032,22841357,"with HIV on HAART, HCV cirrhosis with ascites ..."
5,10000117,22927623,Ms. is a with history of GERD who presents w...
6,10000117,27988844,The patient presented to the emergency departm...
7,10000248,20600184,Mr. is a with history of mild FVIII deficien...
...,...,...,...
331787,19999784,24935234,Mr. is a without significant medical history...
331789,19999828,29734428,Ms. is a year old female with a history of m...
331791,19999840,26071774,year old man hx of epilepsy and left temporal ...
331792,19999840,21033226,Mr. was admitted with status epilepticus. Hi...


In [11]:
discharge.to_csv(os.path.join(output_path, 'discharge_v2_tmp.csv'), index=False)

## normalize lab

In [6]:
labevents = pd.read_csv(os.path.join(output_path, 'labevents_tmp.csv'))
labevents

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
0,10000032,22595853.0,137.0,51491,6.0,NaN,-0.164731
1,10000032,22595853.0,479.0,51221,37.6,NaN,1.116106
2,10000032,22595853.0,479.0,51265,71.0,abnormal,-1.137077
3,10000032,22595853.0,520.0,50882,28.0,NaN,0.572836
4,10000032,22595853.0,520.0,50885,1.3,NaN,-0.205951
...,...,...,...,...,...,...,...
14606355,19999987,23865745.0,9201.0,50912,0.9,NaN,-0.347814
14606356,19999987,23865745.0,9201.0,50971,3.4,NaN,-1.244666
14606357,19999987,23865745.0,9201.0,50983,144.0,NaN,1.178586
14606358,19999987,23865745.0,9208.0,51221,36.6,NaN,0.948310


In [7]:
labevents.valuenum.isna().sum()

0

In [8]:
labevents["normalized_valuenum"] = labevents.groupby('itemid')['valuenum'].transform(lambda x: (x - x.mean()) / x.std())
labevents

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
0,10000032,22595853.0,137.0,51491,6.0,NaN,-0.164731
1,10000032,22595853.0,479.0,51221,37.6,NaN,1.116106
2,10000032,22595853.0,479.0,51265,71.0,abnormal,-1.137077
3,10000032,22595853.0,520.0,50882,28.0,NaN,0.572836
4,10000032,22595853.0,520.0,50885,1.3,NaN,-0.205951
...,...,...,...,...,...,...,...
14606355,19999987,23865745.0,9201.0,50912,0.9,NaN,-0.347814
14606356,19999987,23865745.0,9201.0,50971,3.4,NaN,-1.244666
14606357,19999987,23865745.0,9201.0,50983,144.0,NaN,1.178586
14606358,19999987,23865745.0,9208.0,51221,36.6,NaN,0.948310


In [9]:
labevents.normalized_valuenum.isna().sum()

5

In [10]:
labevents[labevents.normalized_valuenum.isna()]

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
4772841,13270054,22625827.0,29470.0,51266,0.0,abnormal,NaN
7728034,15274195,27623566.0,38289.0,51240,8.0,NaN,NaN
8852725,16034726,29895434.0,12137.0,51266,0.0,abnormal,NaN
10204344,16982199,21508613.0,10853.0,51240,8.0,NaN,NaN
12575673,18600028,27064005.0,2489.0,51021,0.6,NaN,NaN


In [11]:
labevents[labevents.itemid == 51266]

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
4772841,13270054,22625827.0,29470.0,51266,0.0,abnormal,NaN
8852725,16034726,29895434.0,12137.0,51266,0.0,abnormal,NaN


In [12]:
labevents[labevents.itemid == 51240]

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
7728034,15274195,27623566.0,38289.0,51240,8.0,NaN,NaN
10204344,16982199,21508613.0,10853.0,51240,8.0,NaN,NaN


In [13]:
labevents[labevents.itemid == 51021]

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
12575673,18600028,27064005.0,2489.0,51021,0.6,NaN,NaN


In [16]:
labevents = labevents.dropna(subset=["normalized_valuenum"])
labevents

,subject_id,hadm_id,timestamp,itemid,valuenum,flag,normalized_valuenum
0,10000032,22595853.0,137.0,51491,6.0,NaN,-0.164731
1,10000032,22595853.0,479.0,51221,37.6,NaN,1.116106
2,10000032,22595853.0,479.0,51265,71.0,abnormal,-1.137077
3,10000032,22595853.0,520.0,50882,28.0,NaN,0.572836
4,10000032,22595853.0,520.0,50885,1.3,NaN,-0.205951
...,...,...,...,...,...,...,...
14606355,19999987,23865745.0,9201.0,50912,0.9,NaN,-0.347814
14606356,19999987,23865745.0,9201.0,50971,3.4,NaN,-1.244666
14606357,19999987,23865745.0,9201.0,50983,144.0,NaN,1.178586
14606358,19999987,23865745.0,9208.0,51221,36.6,NaN,0.948310


In [17]:
labevents.to_csv(os.path.join(output_path, 'labevents_tmp.csv'), index=False)